<a href="https://colab.research.google.com/github/kato-taki/optimization-of-cultured-meat/blob/main/Control_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input part

## SA module install

In [89]:
#pip install SALib

## color map module install

In [90]:
#pip install bioinfokit

##import module

In [91]:
import numpy as np
import matplotlib.pyplot as plot
#from SALib.sample import saltelli
#from SALib.analyze import sobol
#from bioinfokit import analys, visuz
import pandas as pd
import math
import time
import random
import statistics
import csv

## input value

In [92]:
#2.1 Initial value of parameters

#動物細胞培養
#播種密度
#1サイクル目
X0_cell = 79126666 #cells/L
#最大比増殖速度
umax_cell = 0.02074 #h-1
#培地濃度
Cglc0_cell = 22.75805413 #mmol/L
Clac0_cell = 4.688425104 #mmol/L
Cgln0_cell = 1.789033333 #mmol/L
Camm0_cell = 1.244788914*2 #mmol/L
#培地成分（グルコース，乳酸，グルタミン，アンモニア）のΔC/ΔX（増殖した細胞数に対する生成もしくは消費された培地成分の物質量）
qglc_cell = 0.0000000519922   # ΔCglc/ΔX (mmol/cells)
qlac_cell = 0.0000000908415   # ΔClac/ΔX (mmol/cells)
qgln_cell = 0.00000000395034  # ΔCgln/ΔX (mmol/cells)
qamm_cell = 0.000000000256415 # ΔClac/ΔX (mmol/cells)

#微細藻類培養
#播種密度
#1サイクル目
X0_algae = 4000000000  #cells/L
#最大比増殖速度
umax_algae = 0.01246087  #h-1
#培地濃度
Cglc0_algae = 22.75805413 #mmol/L
Clac0_algae = 27.50518058 #mmol/L
Cgln0_algae = 1.789033333 #mmol/L
Camm0_algae = 1.244788914*2 #mmol/L
#培地成分（乳酸，アンモニア，グルタミン）のΔC/ΔX（増殖した細胞数に対する生成された成分の物質量），グルコースは無視できると仮定
qlac_algae = 0.000000000908281    # ΔClac/ΔX (mmol/cells)
qgln_algae = 0.0000000000189265  # ΔCgln/ΔX (mmol/cells)
qamm_algae = 0.000000000082387    # ΔClac/ΔX (mmol/cells)

#藻類栄養素抽出
#酸加水分解によるグルコース抽出定数
kglc = 0.00000000105492  #mmol/cells

#細胞（培地）抜き取り操作
r = 0.5

In [93]:
#2.2 Standard deviation(SD) of input parameters

#動物細胞培養
#播種密度
SD_X0_cell =X0_cell*0.166
#最大比増殖速度
SD_umax_cell = umax_cell*0.122
#主要成分のΔC/ΔX
SD_qglc_cell = qglc_cell*1.67e-2
SD_qgln_cell = qgln_cell*3.47e-2
SD_qlac_cell = qlac_cell*2.30e-2
SD_qamm_cell = qamm_cell*0.161


#微細藻類培養
#播種密度
SD_X0_algae = X0_algae*1.40e-2
#最大比増殖速度
SD_umax_algae = umax_algae*6.08e-2
#主要成分のΔC/ΔX
SD_qlac_algae = qlac_algae*5.83e-2
SD_qamm_algae = qamm_algae*6.83e-2
SD_qgln_algae = qgln_algae*3.93


#微細藻類酸加水分解
#酸加水分解によるグルコース抽出定数
SD_kglc = kglc*0.1

#細胞（培地）抜き取り操作
SD_r = r*0.001

#液性因子濃度
#動物細胞
SD_Cglc0_cell = Cglc0_cell*1.44e-2
SD_Cgln0_cell = Cgln0_cell*3.01e-2
SD_Clac0_cell = Clac0_cell*0.123
SD_Camm0_cell = Camm0_cell*5.09e-2

#微細藻類
SD_Cglc0_algae = Cglc0_algae*0.114
SD_Clac0_algae = Clac0_algae*6.22e-2
SD_Cgln0_algae = Cgln0_algae*2.60e-2
SD_Camm0_algae = Camm0_algae*5.80e-2

#2.2 coefficient of variation(CV) of input parameters
#液性因子濃度
#動物細胞
CV_Cglc0_cell = 1.44e-2
CV_Cgln0_cell = 3.01e-2
CV_Clac0_cell = 0.123
CV_Camm0_cell = 5.09e-2

#微細藻類
CV_Cglc0_algae = 0.114
CV_Clac0_algae = 6.22e-2
CV_Cgln0_algae = 2.60e-2
CV_Camm0_algae = 5.80e-2

In [94]:
#2.3 Initial value of constants (kinetic equation)
#飽和定数KS
#動物細胞
Kglc = 0.75   #mmol/L
Kgln = 0.075  #mmol/L
#微細藻類
Kamm_algae = 0.0135 #mmol/L
#阻害定数KI
Klac_cell = 172   #mmol/L
Kamm_cell = 28.48 #mmol/L

#乳酸取り込み定数
uptake_max = 0.0000000156 #mmol/L
Klac_uptake = 1.31 #mmol/L
#グルタミン分解定数，
kd_gln = 0.000096 #h-1

In [95]:
#2.4 Setting of section splitting
T = 48
n = 2880
h = T / n
t = np.arange(0,T,h)
run_num = 10
cycle_num = 0

## definition of function

In [96]:
#3
#比増殖速度
#動物細胞
def get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell):
	u = input_umax_cell *(Cglc_cell/(Kglc+Cglc_cell*(1+Clac_cell/Klac_cell)))*(Cgln_cell/(Kgln+Cgln_cell*(1+Camm_cell/Kamm_cell)))
	return u
#藻類
def get_u_algae(input_umax_algae,Camm_algae):
	u= input_umax_algae *(Camm_algae/(Kamm_algae+Camm_algae))
	return u

# Process part

## 1st cycle

### definition of sobol' SA function


In [97]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [98]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 0


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [99]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#累積細胞数のリスト
output_N = np.empty(run_num)
output_N_CV = np.empty(run_num)
#細胞密度，比増殖速度，培地濃度のタイムプロファイルを1つのリストに格納する．
#Mean value
X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t = [],[],[],[],[],[],[]
#CV
X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t = [],[],[],[],[],[]
#格納するクラス
class storage:
  def extend_mean(self,a,b,c,d,e,f,g,h,i,j,k,l,m,n):
    return a.extend(h),b.extend(i),c.extend(j),d.extend(k),e.extend(l),f.extend(m),g.extend(n)

  def extend_CV(self,a,b,c,d,e,f,g,h,i,j,k,l):
    return a.extend(g),b.extend(h),c.extend(i),d.extend(j),e.extend(k),f.extend(l)

storage = storage()
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)

#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1

#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [100]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 2nd cycle

### definition of sobol' SA function


In [101]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [102]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 48


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [103]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [104]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 3rd cycle

### definition of sobol' SA function


In [105]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [106]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 96


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [107]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [108]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 4th cycle

### definition of sobol' SA function


In [109]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [110]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 48*3


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [111]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [112]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 5th cycle

### definition of sobol' SA function


In [113]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [114]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 48*4


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [115]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [116]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 6th cycle

### definition of sobol' SA function


In [117]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [118]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 48*5


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [119]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [120]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 7th cycle

### definition of sobol' SA function


In [121]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [122]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 48*6


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [123]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [124]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 8th cycle

### definition of sobol' SA function


In [125]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [126]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 48*7


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [127]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [128]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 9th cycle

### definition of sobol' SA function


In [129]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [130]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 48*8


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [131]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [132]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

## 10th cycle

### definition of sobol' SA function


In [133]:
#3. generating of random variation of input
size = 1000
param_values = [[random.gauss(X0_cell, SD_X0_cell)for i in range(size)],
    [random.gauss(X0_algae, SD_X0_algae)for i in range(size)],
    [random.gauss(umax_cell, SD_umax_cell)for i in range(size)],
    [random.gauss(umax_algae, SD_umax_algae)for i in range(size)],
    [random.gauss(Cglc0_cell, SD_Cglc0_cell)for i in range(size)],
    [random.gauss(Cglc0_algae, SD_Cglc0_algae)for i in range(size)],
    [random.gauss(Clac0_cell,SD_Clac0_cell)for i in range(size)],
    [random.gauss(Clac0_algae,SD_Clac0_algae)for i in range(size)],
    [random.gauss(Cgln0_cell, SD_Cgln0_cell)for i in range(size)],
    [random.gauss(Cgln0_algae, SD_Cgln0_algae)for i in range(size)],
    [random.gauss(Camm0_cell, SD_Camm0_cell)for i in range(size)],
    [random.gauss(Camm0_algae, SD_Camm0_algae)for i in range(size)],
    [random.gauss(qglc_cell, SD_qglc_cell)for i in range(size)],
    [random.gauss(qlac_cell, SD_qlac_cell)for i in range(size)],
    [random.gauss(qlac_algae, SD_qlac_algae)for i in range(size)],
    [random.gauss(qgln_cell, SD_qgln_cell)for i in range(size)],
    [random.gauss(qgln_algae, SD_qgln_algae)for i in range(size)],
    [random.gauss(qamm_cell, SD_qamm_cell)for i in range(size)],
    [random.gauss(qamm_algae, SD_qamm_algae)for i in range(size)],
    [random.gauss(kglc, SD_kglc)for i in range(size)],
    [random.gauss(r,SD_r)for i in range(size)]]

### model definition

In [134]:
#4. running model and generating output sample
def CCC_system():
  #output
  list_N = np.empty(size) #動物細胞数
  #動物細胞
  list_X_cell_i     = np.empty(size) #次のインプットの細胞密度
  list_Cglc_cell_i  = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_cell_i  = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_cell_i  = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_cell_i  = np.empty(size) #次のインプットのアンモニア濃度
  #微細藻類
  list_X_algae_i    = np.empty(size) #次のインプットの細胞密度
  list_Cglc_algae_i = np.empty(size) #次のインプットのグルコース濃度
  list_Clac_algae_i = np.empty(size) #次のインプットの乳酸濃度
  list_Cgln_algae_i = np.empty(size) #次のインプットのグルタミン濃度
  list_Camm_algae_i = np.empty(size) #次のインプットのアンモニア濃度
  #time profile
  list_X_cell_t     = [[]for i in range(size)] #動物細胞密度
  list_u_cell_t     = [[]for i in range(size)] #動物細胞比増殖速度
  list_Cglc_t       = [[]for i in range(size)] #グルコース濃度
  list_Clac_t       = [[]for i in range(size)] #乳酸濃度
  list_Cgln_t       = [[]for i in range(size)] #グルタミン濃度
  list_Camm_t       = [[]for i in range(size)] #アンモニア
  list_time_t       = [[]for i in range(size)] #時間

  #Process
  #酸加水分解反応後の培地濃度
  list_Cglc_extraction = np.empty(size) #抜き取った培地にグルコースを添加するため，抜き取った培地と元の培地では濃度が異なるから
  list_Cglc_hydrolysis = np.empty(size) #抽出液によってグルコース以外の濃度は変化しないと仮定
  list_Clac_hydrolysis = np.empty(size)
  list_Cgln_hydrolysis = np.empty(size)
  list_Camm_hydrolysis = np.empty(size)
  #培養後の培地濃度
  list_Cglc_exchange = np.empty(size)
  list_Clac_exchange = np.empty(size)
  list_Cgln_exchange = np.empty(size)
  list_Camm_exchange = np.empty(size)

  for q in range(size):

    #Input with variation
    input_X0_cell =     param_values[0][q]
    input_X0_algae =    param_values[1][q]
    input_umax_cell =   param_values[2][q]
    input_umax_algae =  param_values[3][q]
    Cglc_cell =         param_values[4][q]
    Cglc_algae =        param_values[5][q]
    Clac_cell =         param_values[6][q]
    Clac_algae =        param_values[7][q]
    Cgln_cell =         param_values[8][q]
    Cgln_algae =        param_values[9][q]
    Camm_cell =         param_values[10][q]
    Camm_algae =        param_values[11][q]
    input_qglc_cell =   param_values[12][q]
    input_qlac_cell =   param_values[13][q]
    input_qlac_algae =  param_values[14][q]
    input_qgln_cell =   param_values[15][q]
    input_qgln_algae =  param_values[16][q]
    input_qamm_cell =   param_values[17][q]
    input_qamm_algae =  param_values[18][q]
    input_kglc =        param_values[19][q]
    input_r =           param_values[20][q]


    #増殖速度式
    #動物細胞
    f = lambda X_cell,t=0 : u_cell*X_cell
    #藻類
    g = lambda X_algae,t=0 : u_algae*X_algae
    #グルタミンの分解速度式
    gln_cell = lambda Cgln_cell,t=0 : -kd_gln*Cgln_cell
    gln_algae = lambda Cgln_algae,t=0 : -kd_gln*Cgln_algae

    #time profile
    #細胞密度
    #動物細胞
    ls_X_cell = np.empty(n)
    ls_X_cell[0] = input_X0_cell
    #藻類
    ls_X_algae = np.empty(n)
    ls_X_algae[0] = input_X0_algae

    #比増殖速度
    #動物細胞
    ls_u_cell = np.empty(n)
    ls_u_cell[0] = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
    #藻類
    ls_u_algae = np.empty(n)
    ls_u_algae[0] = get_u_algae(input_umax_algae,Camm_algae)

    #培地濃度
    #グルコース濃度
    #動物細胞
    ls_Cglc_cell = np.empty(n)
    ls_Cglc_cell[0] = Cglc_cell
    #藻類
    ls_Cglc_algae = np.empty(n)
    ls_Cglc_algae[0] = Cglc_algae
    #乳酸濃度
    #動物細胞
    ls_Clac_cell = np.empty(n)
    ls_Clac_cell[0] = Clac_cell
    #藻類
    ls_Clac_algae = np.empty(n)
    ls_Clac_algae[0] = Clac_algae
    #アンモニア濃度
    #動物細胞
    ls_Camm_cell = np.empty(n)
    ls_Camm_cell[0] = Camm_cell
    #藻類
    ls_Camm_algae = np.empty(n)
    ls_Camm_algae[0] = Camm_algae
    #グルタミン濃度
    #動物細胞
    ls_Cgln_cell = np.empty(n)
    ls_Cgln_cell[0] = Cgln_cell
    #藻類
    ls_Cgln_algae = np.empty(n)
    ls_Cgln_algae[0] = Cgln_algae

    #Culture time
    ls_time = np.empty(n)
    ls_time[0] = 48*9


    for j in range(0,n-1):
      #比増殖速度
      u_cell = get_u_cell(input_umax_cell,Cglc_cell,Clac_cell,Cgln_cell,Camm_cell)
      u_algae = get_u_algae(input_umax_algae,Camm_algae)

      #1. 動物細胞および藻類増殖時
      if u_cell > input_umax_cell/10 and u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #1.1 動物細胞培養
        #細胞密度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #1.2 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #2. 藻類のみ増殖時
      elif u_algae > input_umax_algae/10 and Camm_cell > 0 and Camm_algae > 0:
        #2.1 藻類培養
        #細胞密度
        j_1 = h * g(ls_X_algae[j],t[j])
        j_2 = h * g(ls_X_algae[j] + j_1 /2 , t[j] + h/2 )
        j_3 = h * g(ls_X_algae[j] + j_2 /2 , t[j] + h/2 )
        j_4 = h * g(ls_X_algae[j] + j_3 , t[j] + h )
        ls_X_algae[j+1] = ls_X_algae[j] + 1/6 * (j_1 + 2*j_2 + 2*j_3 + j_4 )
        #培地濃度
        #グルコース
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae -= input_qlac_algae*u_algae*ls_X_algae[j+1]*h
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        i_1 = h * gln_algae(ls_Cgln_algae[j],t[j])
        i_2 = h * gln_algae(ls_Cgln_algae[j] + i_1 /2 , t[j] + h/2 )
        i_3 = h * gln_algae(ls_Cgln_algae[j] + i_2 /2 , t[j] + h/2 )
        i_4 = h * gln_algae(ls_Cgln_algae[j] + i_3 , t[j] + h )
        ls_Cgln_algae[j+1] = ls_Cgln_algae[j] + 1/6 * (i_1 + 2*i_2 + 2*i_3 + i_4 )+input_qgln_algae*u_algae*ls_X_algae[j+1]*h
        Cgln_algae = ls_Cgln_algae[j+1]
        #アンモニア
        Camm_algae -= input_qamm_algae*u_algae*ls_X_algae[j+1]*h-kd_gln*Cgln_algae*h
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #2.2 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #Culture time
        ls_time[j+1] = ls_time[j] + h


      #3. 動物細胞のみ増殖時
      elif u_cell > input_umax_cell/10 and Camm_cell > 0 and Camm_algae > 0:
        #3.1 動物細胞培養
        #細胞濃度
        k_1 = h * f(ls_X_cell[j],t[j])
        k_2 = h * f(ls_X_cell[j] + k_1 /2 , t[j] + h/2 )
        k_3 = h * f(ls_X_cell[j] + k_2 /2 , t[j] + h/2 )
        k_4 = h * f(ls_X_cell[j] + k_3 , t[j] + h )
        ls_X_cell[j+1] = ls_X_cell[j] + 1/6 * (k_1 + 2*k_2 + 2*k_3 + k_4 )
        #培地濃度
        #グルコース
        Cglc_cell -= input_qglc_cell*u_cell*ls_X_cell[j+1]*h
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell += input_qlac_cell*u_cell*ls_X_cell[j+1]*h
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        h_1 = h * gln_cell(ls_Cgln_cell[j],t[j])
        h_2 = h * gln_cell(ls_Cgln_cell[j] + h_1 /2 , t[j] + h/2 )
        h_3 = h * gln_cell(ls_Cgln_cell[j] + h_2 /2 , t[j] + h/2 )
        h_4 = h * gln_cell(ls_Cgln_cell[j] + h_3 , t[j] + h )
        ls_Cgln_cell[j+1] = ls_Cgln_cell[j] + 1/6 * (h_1 + 2*h_2 + 2*h_3 + h_4 )-input_qgln_cell*u_cell*ls_X_cell[j+1]*h
        Cgln_cell = ls_Cgln_cell[j+1]
        #アンモニア
        Camm_cell += input_qamm_cell*u_cell*ls_X_cell[j+1]*h+kd_gln*Cgln_cell*h
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #3.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

      else:
        #4.1 動物細胞培養
        #細胞密度
        ls_X_cell[j+1] = ls_X_cell[j]
        #培地濃度
        #グルコース
        Cglc_cell = ls_Cglc_cell[j]
        ls_Cglc_cell[j+1] = Cglc_cell
        #乳酸
        Clac_cell = ls_Clac_cell[j]
        ls_Clac_cell[j+1] = Clac_cell
        #グルタミン
        Cgln_cell = ls_Cgln_cell[j]
        ls_Cgln_cell[j+1] = Cgln_cell
        #アンモニア
        Camm_cell = ls_Camm_cell[j]
        ls_Camm_cell[j+1] = Camm_cell
        #比増殖速度
        ls_u_cell[j+1] = u_cell

        #4.2 藻類培養
        #細胞密度
        ls_X_algae[j+1] = ls_X_algae[j]
        #培地濃度
        #グルコース
        Cglc_algae = ls_Cglc_algae[j]
        ls_Cglc_algae[j+1] = Cglc_algae
        #乳酸
        Clac_algae = ls_Clac_algae[j]
        ls_Clac_algae[j+1] = Clac_algae
        #グルタミン
        Cgln_algae = ls_Cgln_algae[j]
        ls_Cgln_algae[j+1] = Cgln_algae
        #アンモニア
        Camm_algae = ls_Camm_algae[j]
        ls_Camm_algae[j+1] = Camm_algae
        #比増殖速度
        ls_u_algae[j+1] = u_algae

        #Culture time
        ls_time[j+1] = ls_time[j] + h

    #time profile
    list_X_cell_t[q] = ls_X_cell
    list_u_cell_t[q] = ls_u_cell
    list_Cglc_t[q] = ls_Cglc_cell
    list_Clac_t[q] = ls_Clac_cell
    list_Cgln_t[q] = ls_Cgln_cell
    list_Camm_t[q] = ls_Camm_cell
    list_time_t[q] = ls_time

    #酸加水分解操作，抜き取り操作
    #1. 酸加水分解操作(藻類栄養素抽出)
    list_Cglc_extraction[q] = Cglc_algae + 5/100*input_kglc*ls_X_algae[-1]*r #抜き取った藻類培地
    list_Cglc_hydrolysis[q] = Cglc_algae #残りの藻類培地
    list_Clac_hydrolysis[q] = Clac_algae
    list_Cgln_hydrolysis[q] = Cgln_algae
    list_Camm_hydrolysis[q] = Camm_algae
    #培地交換前の培地濃度をリストに格納（動物細胞培地）
    list_Cglc_exchange[q] = Cglc_cell
    list_Clac_exchange[q] = Clac_cell
    list_Cgln_exchange[q] = Cgln_cell
    list_Camm_exchange[q] = Camm_cell

    #2. 抜き取り操作
    list_N[q] = ls_X_cell[-1] * r

    #培地交換
    #動物細胞
    Cglc_cell = (1-r)*list_Cglc_exchange[q] + r*list_Cglc_extraction[q]
    Clac_cell = (1-r)*list_Clac_exchange[q] + r*list_Clac_hydrolysis[q]
    Cgln_cell = (1-r)*list_Cgln_exchange[q] + r*list_Cgln_hydrolysis[q]
    Camm_cell = (1-r)*list_Camm_exchange[q] + r*list_Camm_hydrolysis[q]

    #藻類
    Cglc_algae = (1-r)*list_Cglc_hydrolysis[q]  + r*list_Cglc_exchange[q]
    Clac_algae = (1-r)*list_Clac_hydrolysis[q]  + r*list_Clac_exchange[q]
    Cgln_algae = (1-r)*list_Cgln_hydrolysis[q]  + r*list_Cgln_exchange[q]
    Camm_algae = (1-r)*list_Camm_hydrolysis[q]  + r*list_Camm_exchange[q]

    #output
    list_X_cell_i[q] = ls_X_cell[-1]*(1-r)
    list_X_algae_i[q] = ls_X_algae[-1]*(1-r)
    list_Cglc_cell_i[q] = Cglc_cell
    list_Clac_cell_i[q] = Clac_cell
    list_Cgln_cell_i[q] = Cgln_cell
    list_Camm_cell_i[q] = Camm_cell
    list_Cglc_algae_i[q] = Cglc_algae
    list_Clac_algae_i[q] = Clac_algae
    list_Cgln_algae_i[q] = Cgln_algae
    list_Camm_algae_i[q] = Camm_algae

  #output
  N = statistics.mean(list_N)
  N_CV = statistics.stdev(list_N)/statistics.mean(list_N)

  #output(next input)
  #細胞密度
  X_mean = statistics.mean(list_X_cell_i)
  X_sd = statistics.stdev(list_X_cell_i)
  X_algae_mean = statistics.mean(list_X_algae_i)
  X_algae_sd = statistics.stdev(list_X_algae_i)
  #培地濃度
  list_medium = [statistics.mean(list_Cglc_cell_i),statistics.mean(list_Clac_cell_i),statistics.mean(list_Cgln_cell_i),statistics.mean(list_Camm_cell_i),
              statistics.mean(list_Cglc_algae_i),statistics.mean(list_Clac_algae_i),statistics.mean(list_Cgln_algae_i),statistics.mean(list_Camm_algae_i),
              statistics.stdev(list_Cglc_cell_i),statistics.stdev(list_Clac_cell_i),statistics.stdev(list_Cgln_cell_i),statistics.stdev(list_Camm_cell_i),
              statistics.stdev(list_Cglc_algae_i),statistics.stdev(list_Clac_algae_i),statistics.stdev(list_Cgln_algae_i),statistics.stdev(list_Camm_algae_i)]

  #output(time profile)
  ls_X_cell_t = np.empty(n)
  ls_X_cell_CV_t = np.empty(n)
  ls_u_cell_t = np.empty(n)
  ls_u_cell_CV_t = np.empty(n)
  ls_Cglc_cell_t = np.empty(n)
  ls_Cglc_cell_CV_t = np.empty(n)
  ls_Clac_cell_t = np.empty(n)
  ls_Clac_cell_CV_t = np.empty(n)
  ls_Cgln_cell_t = np.empty(n)
  ls_Cgln_cell_CV_t = np.empty(n)
  ls_Camm_cell_t = np.empty(n)
  ls_Camm_cell_CV_t = np.empty(n)
  ls_time_t = np.empty(n)


  for i in range(n):
    ls_X_cell_t[i] = statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_X_cell_CV_t[i] = statistics.stdev(list_X_cell_t[j][i]for j in range(size))/statistics.mean(list_X_cell_t[j][i]for j in range(size))
    ls_u_cell_t[i] = statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_u_cell_CV_t[i] = statistics.stdev(list_u_cell_t[j][i]for j in range(size))/statistics.mean(list_u_cell_t[j][i]for j in range(size))
    ls_Cglc_cell_t[i] = statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Cglc_cell_CV_t[i] = statistics.stdev(list_Cglc_t[j][i]for j in range(size))/statistics.mean(list_Cglc_t[j][i]for j in range(size))
    ls_Clac_cell_t[i] = statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Clac_cell_CV_t[i] = statistics.stdev(list_Clac_t[j][i]for j in range(size))/statistics.mean(list_Clac_t[j][i]for j in range(size))
    ls_Cgln_cell_t[i] = statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Cgln_cell_CV_t[i] = statistics.stdev(list_Cgln_t[j][i]for j in range(size))/statistics.mean(list_Cgln_t[j][i]for j in range(size))
    ls_Camm_cell_t[i] = statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_Camm_cell_CV_t[i] = statistics.stdev(list_Camm_t[j][i]for j in range(size))/statistics.mean(list_Camm_t[j][i]for j in range(size))
    ls_time_t[i] = statistics.mean(list_time_t[j][i]for j in range(size))

  return N,N_CV,X_mean,X_sd,X_algae_mean,X_algae_sd,list_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t

### model output

In [135]:
N,N_CV,X0_cell,SD_X0_cell,X0_algae,SD_X0_algae,result_medium,ls_X_cell_t,ls_X_cell_CV_t,ls_u_cell_t,ls_u_cell_CV_t,ls_Cglc_cell_t,ls_Cglc_cell_CV_t,ls_Clac_cell_t,ls_Clac_cell_CV_t,ls_Cgln_cell_t,ls_Cgln_cell_CV_t,ls_Camm_cell_t,ls_Camm_cell_CV_t,ls_time_t = CCC_system()

#細胞密度，比増殖速度，培地濃度
storage.extend_mean(X_cell_t,u_cell_t,Cglc_cell_t,Clac_cell_t,Cgln_cell_t,Camm_cell_t,time_t,ls_X_cell_t,ls_u_cell_t,ls_Cglc_cell_t,ls_Clac_cell_t,ls_Cgln_cell_t,ls_Camm_cell_t,ls_time_t)
storage.extend_CV(X_cell_CV_t,u_cell_CV_t,Cglc_cell_CV_t,Clac_cell_CV_t,Cgln_cell_CV_t,Camm_cell_CV_t,ls_X_cell_CV_t,ls_u_cell_CV_t,ls_Cglc_cell_CV_t,ls_Clac_cell_CV_t,ls_Cgln_cell_CV_t,ls_Camm_cell_CV_t)
#累積細胞数
output_N[cycle_num] = N
output_N_CV[cycle_num] = N_CV
cycle_num += 1
#培地濃度
#next input value
Cglc0_cell = result_medium[0]
Clac0_cell = result_medium[1]
Cgln0_cell = result_medium[2]
Camm0_cell = result_medium[3]
Cglc0_algae = result_medium[4]
Clac0_algae = result_medium[5]
Cgln0_algae = result_medium[6]
Camm0_algae = result_medium[7]

#next input standard deviation
SD_Cglc0_cell = result_medium[8]
SD_Clac0_cell = result_medium[9]
SD_Cgln0_cell = result_medium[10]
SD_Camm0_cell = result_medium[11]
SD_Cglc0_algae = result_medium[12]
SD_Clac0_algae = result_medium[13]
SD_Cgln0_algae = result_medium[14]
SD_Camm0_algae = result_medium[15]

### sensitivity analysis (ranking system)

In [136]:
# Read samples Y
#Y = np.array(result_X)


# Perform analysis (auto display)
#Si = sobol.analyze(inputs, Y, print_to_console=True)


#Si.plot()
#plot.show()

# Output part

## Time profile

In [137]:
#1. Output of X
with open("./cell density.csv", "w") as f:
    f.write(f"t,mean value,CV\n")
    for i in range(len(X_cell_t)):
      f.write(f"{time_t[i]},{X_cell_t[i]},{X_cell_CV_t[i]}\n")

In [138]:
#2. Output of μ
with open("./specific growth rate.csv", "w") as f:
    f.write(f"t,mean value,CV\n")
    for i in range(len(u_cell_t)):
      f.write(f"{time_t[i]},{u_cell_t[i]},{u_cell_CV_t[i]}\n")

In [139]:
#3. Output of medium
with open("./medium.csv", "w") as f:
    f.write(f"t,Cglc,CV,Clac,CV,Cgln,CV,Camm,CV\n")
    for i in range(len(Cglc_cell_t)):
      f.write(f"{time_t[i]},{Cglc_cell_t[i]},{Cglc_cell_CV_t[i]},{Clac_cell_t[i]},{Clac_cell_CV_t[i]},{Cgln_cell_t[i]},{Cgln_cell_CV_t[i]},{Camm_cell_t[i]},{Camm_cell_CV_t[i]}\n")

In [140]:
#4. Output of N
with open("./N.csv", "w") as f:
    f.write(f"1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th\n")
    for i in range(len(output_N)):
      f.write(f"{output_N[i]},")
    f.write("\n")

In [141]:
#5. Output CV of N
with open("./CV of N.csv", "w") as f:
    f.write(f"1st,2nd,3rd,4th,5th,6th,7th,8th,9th,10th\n")
    for i in range(len(output_N_CV)):
      f.write(f"{output_N_CV[i]},")
    f.write("\n")

##Color Map

In [142]:
#df = pd.read_csv('timeprofile (CV of medium).csv', encoding='latin-1')
#df = df.set_index(df.columns[0])
#visuz.gene_exp.hmap(df=df, dim=(3, 6), tickfont=(11, 11), rowclus=False, colclus=False, figname='heatmap2')